<a href="https://colab.research.google.com/github/brittabeta/Data-Engineering-Duke/blob/main/persistence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Persistence and Efficency
persist the collected data, so that it can be analyzed and cleaned later - 
parsing, retrieving, saving, and cleaning data are all separate actions - do NOT try to work with data while collecting. 

* parsing techniques
* persistency: JSON, CSV, SQL and a database

load files and `scrapy` library

In [2]:
# create an activated a virtual environment 
!python3 -m venv venv
!source venv/bin/activate
!pip install -r requirements.txt

Error: Command '['/content/venv/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/bin/bash: venv/bin/activate: No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.0/255.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 8.5 MB/s eta 0:00:00


In [4]:
import scrapy
import os
current_dir = os.path.abspath('')
url = os.path.join(current_dir, "/content/1992_World_Junior_Championships_in_Athletics_–_Men's_high_jump")
with open(url) as _f:
    url_data = _f.read()

response = scrapy.http.TextResponse(url, body=url_data, encoding='utf-8')

In [5]:
# interesting data is available ?
table = response.xpath('//table')[1].xpath('tbody')
for tr in table.xpath('tr'):
    print(tr.xpath('td/b/text()').extract()[0],
          tr.xpath('td/a/text()').extract()[0]
    )

Gold Steve Smith
Silver Tim Forsyth
Bronze Takahiro Kimino


/usr/local/lib/python3.10/dist-packages/scrapy/selector/unified.py:82: UserWarning: Selector got both text and root, root is being ignored.
  super().__init__(text=text, type=st, root=root, **kwargs)


interaction with `scrapy` in a Notebook is useful:
* don't need to run the special shell 
* don't need to run the whole spider
* good testing env to adapt spider 

persisting with JSON
* keep the information in a Python data structure 
* dictionary
* load it as a JSON object
* save it to a file

In [8]:
scrapped_data = {}
for tr in table.xpath('tr'):
    medal = tr.xpath('td/b/text()').extract()[0]
    athlete = tr.xpath('td/a/text()').extract()[0]
    scrapped_data[medal] = athlete

scrapped_data

{'Gold': 'Steve Smith', 'Silver': 'Tim Forsyth', 'Bronze': 'Takahiro Kimino'}

In [9]:
import json

#json_data = json.dumps(scrapped_data)

# persist
with open("1992_results.json", "w") as _f:
    # use dump() with the Python dictionary directly. 
    # the conversion is done on the fly
    json.dump(scrapped_data, _f)

    # creates file /content/1992_results.json with key:value

persist with CSV
* can use Pandas
* can also use the standard library CSV module

In [10]:
# construct the data first

column_names = ["Medal", "Athlete"]
rows = []

for tr in table.xpath('tr'):
    medal = tr.xpath('td/b/text()').extract()[0]
    athlete = tr.xpath('td/a/text()').extract()[0]
    rows.append([medal, athlete])


In [11]:
# persist it to disk
import csv

with open("1992_results.csv", "w") as _f:
    writer = csv.writer(_f)

    # write the column names
    writer.writerow(column_names)

    # now write the rows
    writer.writerows(rows)

persist to a database:
* much more memory efficient
* can save the data as the data is gathered 
* SQLite database to persist the data
* file-based database and the table needed

In [12]:
import sqlite3
connection = sqlite3.connect("1992_results.db")
db_table = 'CREATE TABLE results (id integer primary key, medal TEXT, athlete TEXT)'
cursor = connection.cursor()
cursor.execute(db_table)
connection.commit()

In [13]:
# persist the data
connection = sqlite3.connect("1992_results.db")
cursor = connection.cursor()
query = 'INSERT INTO results(medal, athlete) VALUES(?, ?)'

for tr in table.xpath('tr'):
    medal = tr.xpath('td/b/text()').extract()[0]
    athlete = tr.xpath('td/a/text()').extract()[0]
    cursor.execute(query, (medal, athlete)) 
    connection.commit()

In [24]:
# connect and query
db = "/content/1992_results.db"
connection = sqlite3.connect(db)
cursor = connection.cursor()
query = 'SELECT * FROM results'
cursor.execute(query)
cursor.fetchall() # or fetchone()

[(1, 'Gold', 'Steve Smith'),
 (2, 'Silver', 'Tim Forsyth'),
 (3, 'Bronze', 'Takahiro Kimino')]

In [25]:
# close
connection.close()